In [1]:
from widgetscript import JsContext, pycall 
from yattag import Doc
from IPython.display import display, HTML
import requests
from io import BytesIO
from skimage.io import imread
import base64

In [2]:
def image_bytes_to_img_src(image_bytes):
    return "data:image/png;base64, "+base64.b64encode(image_bytes).decode()

class ColorPicker:
    def __init__(self, image_bytes):
        image = imread(BytesIO(image_bytes))

        ctx = JsContext()

        def int_round(x):
            return int(round(x))

        @ctx.py
        def resolve_color(x, y, image_width, image_height):
            x_scale = image.shape[1]/image_width
            y_scale = image.shape[0]/image_height
            return image[int_round(y*y_scale), int_round(x*x_scale)].tolist()

        @ctx.js
        def find_element(query):
            return jQuery("#color_picker_" + context.id + " "+query)

        @ctx.js
        def on_image_click(e):
            x = e.originalEvent.layerX if e.offsetX == None else e.offsetX;
            y = e.originalEvent.layerY if e.offsetY == None else e.offsetY;
            image = find_element(".color_picker_image")
            pycall(resolve_color(x, y, image.width(), image.height()), on_get_color)

        @ctx.js
        def on_get_color(rgb):
            r, g, b = rgb
            find_element(".color_picker_color").css("background-color","rgb("+r+","+g+","+b+")")
            find_element(".color_picker_rgb").text("R: "+r+", G: "+g+", B: "+b)

        @ctx.js_flat
        def js_init():
            image = find_element(".color_picker_image")
            image.on("click", on_image_click)

        doc, tag, text = Doc().tagtext()

        with tag("div", id="color_picker_" + ctx.id):
            doc.stag("img", src=image_bytes_to_img_src(image_bytes), klass="color_picker_image", 
                     style="max-width: 800px; display:inline-block; cursor: pointer;")
            with tag("div", style="display:inline-block; margin-left: 15px;"):
                doc.stag("div", klass="color_picker_color", style="width:100px; height:100px")
                doc.stag("div", klass="color_picker_rgb")
            
        doc.asis(ctx.html())
        self.html = doc.getvalue()
        
    def _repr_html_(self):
        return self.html

In [3]:
with open("./hummingbird.jpeg", "rb") as f:
    image = f.read()
ColorPicker(image)